# Describe a PIV Result File

In this example, we want to describe the result file of the ILA Vortex pair (source: https://www.pivtec.com/download/samples/VortexPairSeq.zip)

Where to start? Let's first evaluate what we have:

General context:
- The general concept of a dataset is described by `dcat:Dataset`
- The file is described by `pivmeta:PivResultDistribution` and is part of the `dcat:Dataset`

Specific information:<br>
Of greater interest is the PIV process including the PIV parameters leading to the dataset. A `dcat:Dataset` is the output of the PIV process. For this, we can use [`m4i:output of`](http://purl.obolibrary.org/obo/RO_0002353):

In [1]:
from pivmetalib import pivmeta, dcat, m4i, schema, sd, prov

In [2]:
result_dist = pivmeta.PivResultDistribution(
    title='Result File',
    downloadURL='file:///vp1a.dat'
)
# the "downloaded" file must exist:
assert result_dist.download().exists()

In [3]:
result_dist.downloadURL

Url('file:///vp1a.dat')

In [4]:
pivmeta.PivImageType.ExperimentalImage

<PivImageType.ExperimentalImage: rdflib.term.URIRef('https://matthiasprobst.github.io/pivmeta#ExperimentalImage')>

In [5]:
piv_image_dist = pivmeta.PivImageDistribution(
    title='ILA Vortex Pair Images',
    download_URL='https://www.pivtec.com/download/samples/pivimg1.zip',
    piv_image_type=pivmeta.PivImageType.ExperimentalImage
)
piv_image_dist

PivImageDistribution(title=ILA Vortex Pair Images, download_URL=https://www.pivtec.com/download/samples/pivimg1.zip, piv_image_type=PivImageType.ExperimentalImage)

In [6]:
ds = dcat.Dataset(
    title='ILA Vortex Pair',
    distribution=[result_dist, piv_image_dist]
)
ds

Dataset(title=ILA Vortex Pair, distribution=[PivResultDistribution(title=Result File, downloadURL=file:///vp1a.dat), PivImageDistribution(title=ILA Vortex Pair Images, download_URL=https://www.pivtec.com/download/samples/pivimg1.zip, piv_image_type=PivImageType.ExperimentalImage)])

Before defining the PIV processing steps, we need to describe the software used:

In [7]:
piv_software = pivmeta.PIVSoftware(
    author=prov.Organization(
        name='PIVTEC GmbH',
        mbox='info@pivtec.com',
        url='https://www.pivtec.com/'
    ),
    has_documentation='https://www.pivtec.com/download/docs/PIVview_v36_Manual.pdf',
)

In [8]:
# import h5py
# from typing import Dict


# import h5rdmtoolbox as h5tbx
# h5tbx.set_config(natural_naming=False)

# with h5tbx.File() as h5:
#     piv_software.dump_hdf(h5)
#     h5.dump()
#
# print(piv_software.dump_jsonld())

## Processing steps

A `PivProcessingStep` class is provided in order to distinguish the processing step from others. Some methods are provided as classes but without specific properties. This is done to provide flexibility, as all methods can be standardized. However, by introducing **standard names**, the authors may narrow their parameter definitions either by using global standard names (with an IRI) or within their project.

*TODO: Put here an image illustrating the possibilities*

By using standard names, important parameters can be identified unambiguously. See `PIVMETA.image_filter_kernel_size` in the example in contrast to the 180° image rotation.

### 1. Pre-Processing (Image processing)

Methods:
- image rotation by 180 deg

In [9]:
from ontolutils import PIVMETA

In [10]:
pre = pivmeta.PivPreProcessing(
    name='Image pre processing',
    realizesMethod=[
        m4i.Method(
            description='Rotates the input image by 180 deg',
            parameter=m4i.NumericalVariable(
                name='rotation',
                value=180,
                unit='deg',
                quantity_kind='https://qudt.org/vocab/unit/DEG'
            )
        ),
        # Dont define all the classes for filters and outlier detection because everybody may define it differently.
        # common parameters can be specified "on demand" by standard names like so: 
        m4i.Method(
            description='Applies a median filter to the image',
            hasParameter=pivmeta.NumericalVariable(
                standard_name=PIVMETA.image_filter_kernel_size,
                value=3,
            )
        )
    ]
)
pre

PivPreProcessing(realizesMethod=[Method(description=Rotates the input image by 180 deg, parameter=NumericalVariable(value=180, unit=deg, quantity_kind=https://qudt.org/vocab/unit/DEG, name=rotation)), Method(description=Applies a median filter to the image, hasParameter=NumericalVariable(value=3, standard_name=https://matthiasprobst.github.io/pivmeta#image_filter_kernel_size))], name=Image pre processing)

## 2. PIV evaluation

The PIV evaluation is a `m4i:ProcessingStep`. It realizes the methods
- `pivemta:CorrelationMethod` (e.g. FFT, ... ),
- `pivemta:InterrogationMethod` (e.g. `pivemta:Multigrid`, ...) and
- `pivemta:PeakSearchMethod`.

The outlier detection (validation) is a sub-processing step (`pivemta:PivValidation`), because it can realize multiple methods. Most popular ones are described by classes:
- `pivemta:DynMean`.

Here, a multigrid evaluation is performed using standard FFT.

### 2.1 Correlation algorithm

The correlation algorithm is a subclass of `m4i:Method`. At least a name and description should be provided (here, taken from the documentation). We could also provide parameters.

In [11]:
calgo = pivmeta.CorrelationMethod(
    name='Standard (FFT) Correlation',
    windowWeightingFunction='square',
    description='The default mode of cross-correlation using FFTs to speed the computation. ' 
    'In principle the sum of pixel-wise multiplication of intensities is computed for each ' 
    'correlation offset (For implementation details please refer to Raffel et al. 2007).'
)
print(calgo.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "schema": "https://schema.org/",
        "pivmeta": "https://matthiasprobst.github.io/pivmeta#"
    },
    "@type": "pivmeta:CorrelationMethod",
    "schema:description": "The default mode of cross-correlation using FFTs to speed the computation. In principle the sum of pixel-wise multiplication of intensities is computed for each correlation offset (For implementation details please refer to Raffel et al. 2007).",
    "pivmeta:windowWeightingFunction": "https://matthiasprobst.github.io/pivmeta#SquareWindowWeightingFunction",
    "name": "Standard (FFT) Correlation",
    "@id": "N68b6496319304d58b41b8fe9953a2343"
}


### 2.2 Interrogation method

In this example, a *Multi-Grid* method was used starting from a window with size 64 px down to 16 px in 3 steps

In [12]:
v = pivmeta.NumericalVariable(
            standard_name=PIVMETA.x_initial_interrogation_window_size,
            value=64,
        )
print(v.model_dump_jsonld(resolve_keys=False))

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "pivmeta": "https://matthiasprobst.github.io/pivmeta#",
        "value": "http://w3id.org/nfdi4ing/metadata4ing#hasNumericalValue",
        "standard_name": "https://matthiasprobst.github.io/pivmeta#hasStandardName"
    },
    "@type": "m4i:NumericalVariable",
    "value": 64,
    "standard_name": "https://matthiasprobst.github.io/pivmeta#x_initial_interrogation_window_size",
    "@id": "Nd09e1aae6223414f9330045a20af93d4"
}


In [13]:
#print(v.dump_jsonld())

In [14]:
int_meth = m4i.Method(
    name='Multigrid interrogation method',
    description='Run a multigrid PIV algorithm on all images',
    hasParameter=[
        pivmeta.NumericalVariable(
            standard_name=PIVMETA.x_initial_interrogation_window_size,
            value=64,
        ),
        pivmeta.NumericalVariable(
            standard_name=PIVMETA.y_initial_interrogation_window_size,
            value=64,
        ),
        pivmeta.NumericalVariable(
            standard_name=PIVMETA.x_final_interrogation_window_size,
            value=16,
        ),
        pivmeta.NumericalVariable(
            standard_name=PIVMETA.y_final_interrogation_window_size,
            value=16,
        ),
        pivmeta.NumericalVariable(
            standard_name=PIVMETA.x_final_interrogation_window_overlap_size,
            value=8,
        ),
        pivmeta.NumericalVariable(
            standard_name=PIVMETA.y_final_interrogation_window_overlap_size,
            value=8,
        ),
        pivmeta.NumericalVariable(
            # hasStandardName=PIVMETA.number_of_multigrid_passes,
            description='Number of multigrid passes',
            value=3,
        )
    ]
)
int_meth.hasParameter[-1]

NumericalVariable(value=3, description=Number of multigrid passes)

### 2.3 Outlier detection

We use the following two methods for outlier detection:
- normalized median test threshold: 3.0 (see DOI=https://doi.org/10.1007/s00348-005-0016-6)
- dynamic mean test: mean=2.0, var=1.0

In [15]:
median_test = pivmeta.OutlierDetectionMethod(
    name='normalized median test',
    hasParameter=m4i.NumericalVariable(
        label='threshold',
        value=3.0
    )
)

In [16]:
dyn_mean = pivmeta.OutlierDetectionMethod(
    name='dynamic mean test',
    hasParameter=[
        m4i.NumericalVariable(
            label='mean',
            value=2.0
        ),
        m4i.NumericalVariable(
            label='var',
            value=1.0
        )
    ]
)

In [17]:
proc = pivmeta.PivEvaluation(
    label='piv evaluation',
    realizesMethod=[
        calgo,
        int_meth,
        median_test,
        dyn_mean
    ]
)
proc

PivEvaluation(label=piv evaluation, realizesMethod=[CorrelationMethod(description=The default mode of cross-correlation using FFTs to speed the computation. In principle the sum of pixel-wise multiplication of intensities is computed for each correlation offset (For implementation details please refer to Raffel et al. 2007)., windowWeightingFunction=https://matthiasprobst.github.io/pivmeta#SquareWindowWeightingFunction, name=Standard (FFT) Correlation), Method(description=Run a multigrid PIV algorithm on all images, name=Multigrid interrogation method, hasParameter=[NumericalVariable(value=64, standard_name=https://matthiasprobst.github.io/pivmeta#x_initial_interrogation_window_size), NumericalVariable(value=64, standard_name=https://matthiasprobst.github.io/pivmeta#y_initial_interrogation_window_size), NumericalVariable(value=16, standard_name=https://matthiasprobst.github.io/pivmeta#x_final_interrogation_window_size), NumericalVariable(value=16, standard_name=https://matthiasprobst.github.io/pivmeta#y_final_interrogation_window_size), NumericalVariable(value=8, standard_name=https://matthiasprobst.github.io/pivmeta#x_final_interrogation_window_overlap_size), NumericalVariable(value=8, standard_name=https://matthiasprobst.github.io/pivmeta#y_final_interrogation_window_overlap_size), NumericalVariable(value=3, description=Number of multigrid passes)]), OutlierDetectionMethod(, name=normalized median test, hasParameter=NumericalVariable(label=threshold, value=3.0)), OutlierDetectionMethod(, name=dynamic mean test, hasParameter=[NumericalVariable(label=mean, value=2.0), NumericalVariable(label=var, value=1.0)])])

In [18]:
data_smoothing = m4i.Method(
    name='Low-pass filtering',
    description='applies a low-pass filtering on the data using a Gaussian weighted kernel of specified width to reduce spurious noise.',
    parameter=m4i.NumericalVariable(label='kernel', value=2.0)
)

In [19]:
post = pivmeta.PivPostProcessing(
    label='Post processing',
    realizesMethod=data_smoothing
)
post.model_dump_jsonld()         

'{\n    "@context": {\n        "owl": "http://www.w3.org/2002/07/owl#",\n        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",\n        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",\n        "schema": "https://schema.org/",\n        "obo": "http://purl.obolibrary.org/obo/",\n        "pivmeta": "https://matthiasprobst.github.io/pivmeta#"\n    },\n    "@type": "pivmeta:PivProcessingStep",\n    "rdfs:label": "Post processing",\n    "m4i:realizesMethod": {\n        "@type": "m4i:Method",\n        "schema:description": "applies a low-pass filtering on the data using a Gaussian weighted kernel of specified width to reduce spurious noise.",\n        "m4i:hasParameter": {\n            "@type": "m4i:NumericalVariable",\n            "rdfs:label": "kernel",\n            "m4i:hasNumericalValue": 2.0,\n            "@id": "N4eaea58a4a4542489e4f8d915b8e07a4"\n        },\n        "name": "Low-pass filtering",\n        "@id": "Na065edb9a306478bbb64ddfedb76828c"\n    },\n    "@id": "N1dcaa17681a74

## 3. Creating the full Meta document (connect information)

We created three processing steps:
1. pre (takes raw images)
2. proc
3. post (outputs result data)


In [20]:
post.hasOutput = ds

## 4. dump PIV run to JSON-LD

In [21]:
piv = m4i.ProcessingStep(
    label='PIV Run',
    startsWith=pre,
    endsWith=post
)
# proc.part_of = piv
pre.precedes = proc
proc.precedes = post

# all processing steps were employed by software pivview:
pre.has_employed_tool = piv_software
proc.has_employed_tool = piv_software
post.has_employed_tool = piv_software

In [22]:
print(pre.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "schema": "https://schema.org/",
        "obo": "http://purl.obolibrary.org/obo/",
        "pivmeta": "https://matthiasprobst.github.io/pivmeta#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "dcat": "http://www.w3.org/ns/dcat#",
        "dcterms": "http://purl.org/dc/terms/",
        "prov": "http://www.w3.org/ns/prov#",
        "sd": "https://w3id.org/okn/o/sd#",
        "foaf": "http://xmlns.com/foaf/0.1/"
    },
    "@type": "pivmeta:PivPostProcessing",
    "m4i:realizesMethod": [
        {
            "@type": "m4i:Method",
            "schema:description": "Rotates the input image by 180 deg",
            "m4i:hasParameter": {
                "@type": "m4i:NumericalVariable",
                "m4i:hasNumericalValue": 180,
                "m4i:hasUnit": "deg",
             

In [23]:
with open('piv_process.json', 'w') as f:
    f.write(proc.model_dump_jsonld())

In [24]:
print(piv.model_dump_jsonld(
    context={"@import": "https://raw.githubusercontent.com/matthiasprobst/pivmeta/main/pivmeta_context.jsonld"})
     )

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "schema": "https://schema.org/",
        "obo": "http://purl.obolibrary.org/obo/",
        "@import": "https://raw.githubusercontent.com/matthiasprobst/pivmeta/main/pivmeta_context.jsonld",
        "pivmeta": "https://matthiasprobst.github.io/pivmeta#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "dcat": "http://www.w3.org/ns/dcat#",
        "dcterms": "http://purl.org/dc/terms/",
        "prov": "http://www.w3.org/ns/prov#",
        "sd": "https://w3id.org/okn/o/sd#",
        "foaf": "http://xmlns.com/foaf/0.1/"
    },
    "@type": "m4i:ProcessingStep",
    "rdfs:label": "PIV Run",
    "startsWith": {
        "@type": "pivmeta:PivPostProcessing",
        "m4i:realizesMethod": [
            {
                "@type": "m4i:Method",
                "schema:description": "Rotates